In [1]:
import os
from datetime import datetime
#from datetime import date
import pymysql
from sqlalchemy import create_engine
from decouple import config 
from dotenv import load_dotenv
import pandas as pd
import numpy as np

In [2]:
load_dotenv()
# get the environment variables needed
USER= config('USRCaris')
PASSWORD= config('PASSCaris')
HOSTNAME= config('HOSTCaris')
DBNAME= config('DBCaris')

# get the engine to connect and fetch
engine = create_engine(f"mysql+pymysql://{USER}:{PASSWORD}@{HOSTNAME}/{DBNAME}")
query = '''
SELECT
    p.patient_code as code,
    b.first_name,
    b.last_name,
    b.dob,
    dh.name AS hub_name,
    dh.address AS hub_address,
    dh.id as id_hub,
    dg.name AS group_name,
    dg.id as id_group,
    dgs.date AS session_date,
    ldt.name AS topic_name,
	dgs.topic as id_topic,
    ds.value as presence
FROM
    dream_group_attendance ds
        LEFT JOIN
    dream_group_session dgs ON dgs.id = ds.id_group_session
        LEFT JOIN
    dream_group dg ON dg.id = dgs.id_group
        LEFT JOIN
    dream_hub dh ON dh.id = dg.id_dream_hub
        LEFT JOIN
    beneficiary b ON b.id_patient = ds.id_patient
        LEFT JOIN
    lookup_dreams_topic ldt ON ldt.id = dgs.topic
        LEFT JOIN
    patient p ON p.id = ds.id_patient
'''

curriculum_attendance = pd.read_sql_query(query,engine,parse_dates=True)
# get the test excel file from Query

# close the pool of connection
engine.dispose()

In [3]:
curriculum_attendance.shape

(434404, 13)

In [4]:
curriculum_attendance.to_excel('curriculum_attendance.xlsx',index=False,na_rep="NULL")